<a href="https://colab.research.google.com/github/cerule7/Cores-Project/blob/master/Kenosha_Data_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Use the 2020 presidential election data as a template
# 2024 presidential election data will presumably be at https://apps.kenoshacounty.org/ElectionResults_v2/Default.aspx?eid=64
url_to_scrape = 'https://apps.kenoshacounty.org/ElectionResults_v2/Default.aspx?eid=51'

In [24]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [5]:
# Fetch the HTML content from the URL
response = requests.get(url_to_scrape)
html_content = response.content

# Create a BeautifulSoup object to parse the HTML
soup = BeautifulSoup(html_content, 'html.parser')

In [7]:
print(soup.prettify())

<!DOCTYPE html>
<head>
 <title>
  Election Results
 </title>
 <link href="ElectionResults.css" rel="stylesheet"/>
 <script src="Scripts/jquery-2.1.3.js">
 </script>
 <script type="text/javascript">
  function OpenPrintWindow() {
            window.open("Default_print.aspx?eid=51", "_blank", "status=1,toolbar=1,menubar=1,location=0,scrollbars=1");
      }

      function RefreshPage() {
          location.reload(true);
      }

      function jumpRace() {
          var race = $("#ddContests").val();
          if (race != -1) {
              document.getElementById("contest" + race).scrollIntoView(true);
          }
      }
 </script>
</head>
<body>
 <form action="./Default.aspx?eid=51" id="form1" method="post">
  <input id="__VIEWSTATE" name="__VIEWSTATE" type="hidden" value="..."/>
  <input id="__VIEWSTATEGENERATOR" name="__VIEWSTATEGENERATOR" type="hidden" value="29F4C1DC"/>
  <input id="__VIEWSTATEENCRYPTED" name="__VIEWSTATEENCRYPTED" type="hidden" value=""/>
  <input id="__EVENTVAL

In [25]:
# Scrapes all of tables from the page
def parse_contest_boxes(soup):
    contest_boxes = soup.find_all('div', {'class': 'contestBox'})
    dfs = []

    for box in contest_boxes:
        contest_name = box.h1.text.strip()
        poll_info = box.h2.text.strip()

        table = box.find('table', {'class': 'resultTable'})
        if table:
            data = []
            for row in table.find_all('tr'):
                candidate_name = row.find('td', {'class': 'candtd'}).text.strip()
                vote_info = row.find_all('div')[1].text.strip()
                vote_count, vote_percentage = vote_info.split('(')
                vote_percentage = vote_percentage.strip('%)')
                data.append([candidate_name, int(vote_count), float(vote_percentage)])

            df = pd.DataFrame(data, columns=['Candidate', 'Votes', 'Percentage'])
            df['Contest'] = contest_name
            df['Poll Info'] = poll_info
            dfs.append(df)

    return dfs

In [26]:
contest_dfs = parse_contest_boxes(soup)

first_contest_df = contest_dfs[0]
print(first_contest_df)

                                Candidate  Votes  Percentage  \
0  REP Donald J. Trump / Michael R. Pence  44972       50.71   
1  DEM Joseph R. Biden / Kamala D. Harris  42193       47.57   
2   LBP Jo Jorgensen / Jeremy Spike Cohen   1037        1.17   
3                               write-in:    224        0.25   
4      CON Don Blankenship / William Mohr    147        0.17   
5          ASP Brian Carroll / Amar Patel    120        0.14   

                      Contest                         Poll Info  
0  President / Vice President  (140 polls reporting out of 140)  
1  President / Vice President  (140 polls reporting out of 140)  
2  President / Vice President  (140 polls reporting out of 140)  
3  President / Vice President  (140 polls reporting out of 140)  
4  President / Vice President  (140 polls reporting out of 140)  
5  President / Vice President  (140 polls reporting out of 140)  


In [35]:
for df in contest_dfs:
    print(df['Contest'][0])
    print(df[['Candidate', 'Votes', 'Percentage', 'Poll Info']])

President / Vice President
                                Candidate  Votes  Percentage  \
0  REP Donald J. Trump / Michael R. Pence  44972       50.71   
1  DEM Joseph R. Biden / Kamala D. Harris  42193       47.57   
2   LBP Jo Jorgensen / Jeremy Spike Cohen   1037        1.17   
3                               write-in:    224        0.25   
4      CON Don Blankenship / William Mohr    147        0.17   
5          ASP Brian Carroll / Amar Patel    120        0.14   

                          Poll Info  
0  (140 polls reporting out of 140)  
1  (140 polls reporting out of 140)  
2  (140 polls reporting out of 140)  
3  (140 polls reporting out of 140)  
4  (140 polls reporting out of 140)  
5  (140 polls reporting out of 140)  
Representative in Congress District 1
          Candidate  Votes  Percentage                         Poll Info
0   REP Bryan Steil  48524       55.81  (140 polls reporting out of 140)
1  DEM Roger Polack  38337       44.09  (140 polls reporting out of 140)
2